In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import callbacks
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.optimizers import Adam
import time
from keras import regularizers

In [2]:
DATA_INPUT_FILE = 'lichessParser/bitBoard.npy'
BATCH_SIZE = 4096  # Define the batch size

In [3]:
def numStr(num):
    return '{:,}'.format(
        np.round(num, 6)
    ).replace(',', ' ')

print('loading')
mapped_array = np.load(DATA_INPUT_FILE)[:2_000_000]

print('items loaded: ', numStr(mapped_array.shape[0]))

print('processing data')
mapped_array = np.unpackbits(
    mapped_array.view(np.uint8),
    bitorder='little',
    axis=1
).view(bool)[:, :773]

print('Done')

loading
items loaded:  2 000 000
processing data
Done


In [4]:
from sys import getsizeof

print(len(mapped_array))
print(mapped_array.nbytes / 1024/ 1024/ 1024, 'GiB')


2000000
1.4398247003555298 GiB


In [5]:
# Step 3: Define and Train the Encoder-Decoder model
my_regularizer = regularizers.l1(10e-8)

input_layer = Input(shape=(773,), dtype=bool) # 12*64+5
hidden_encoder = Dense(8000, activation='relu')(input_layer)
hidden_encoder2 = Dense(2000, activation='relu')(hidden_encoder)

latent_layer = Dense(10,
                     activation='relu', 
                     #activity_regularizer=my_regularizer
                    )(hidden_encoder2)

hidden_decoder = Dense(2000, activation='relu')(latent_layer)
hidden_decoder2 = Dense(8000, activation='relu')(hidden_decoder)
output_layer = Dense(773, activation='sigmoid')(hidden_decoder2)

In [6]:
# Full model: Encoder + Decoder
autoencoder = Model(inputs=input_layer, outputs=output_layer)
optimizer = Adam(
    learning_rate=0.00001
)
autoencoder.compile(
    optimizer=optimizer,
   # loss='mse'
    #loss=keras.losses.MeanAbsoluteError(),
    loss='binary_crossentropy'
)
print(autoencoder.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 773)]             0         
                                                                 
 dense (Dense)               (None, 8000)              6192000   
                                                                 
 dense_1 (Dense)             (None, 2000)              16002000  
                                                                 
 dense_2 (Dense)             (None, 10)                20010     
                                                                 
 dense_3 (Dense)             (None, 2000)              22000     
                                                                 
 dense_4 (Dense)             (None, 8000)              16008000  
                                                                 
 dense_5 (Dense)             (None, 773)               618477

In [11]:
history = autoencoder.fit(mapped_array, mapped_array, epochs=5)

Epoch 1/5
62500/62500 [==============================] - 405s 6ms/step - loss: 0.0328
Epoch 2/5
62500/62500 [==============================] - 405s 6ms/step - loss: 0.0317
Epoch 3/5
62500/62500 [==============================] - 405s 6ms/step - loss: 0.0307
Epoch 4/5
62500/62500 [==============================] - 405s 6ms/step - loss: 0.0298
Epoch 5/5
62500/62500 [==============================] - 424s 7ms/step - loss: 0.0290


In [8]:
print(history.history['loss'])

[0.050542719662189484, 0.04181637242436409, 0.03842030093073845, 0.03600088879466057, 0.03416614979505539]


In [9]:
# loss=binary_crossentropy
# struct: 8k[relu] -> 2k[relu] -> 100[relu] 
epoch=1  LR=0.00001  loss= 0.05 0.042 0.038 0.036 0.0342 ... 0.029

# struct: 16k[relu] -> 10[relu] 
epoch=4  LR=0.00001  loss= 0.055  0.046  0.0434  0.0411

# struct: 16k[relu] -> 100[relu] 
epoch=2  LR=0.00001  loss=0.0021
epoch=1  LR=0.00001  loss=0.017

# struct: 8k[relu] -> 100[relu] 
epoch=1  LR=0.00001  loss=0.0212

# struct: 8k[relu] -> 100[tanh] 
epoch=1  LR=0.00001  loss=0.0495





SyntaxError: invalid syntax (3694420279.py, line 3)

In [13]:
# SAVE THE MODEL
autoencoder.save('chess_autoencoder.h5')

In [ ]:
autoencoder.predict(np.array([[-1., -0.667, 0., -0.5, -0.833, -1., -0.5, -0.333, -0.667, -0.167
, -0.167, 0., 0., -0.167, -0.167, -0.167, -0.167, 0., 0., -0.333
, 0., 0., 0., 0., 0., 0., 0., 0., -0.167, 0.
, 0., 0., 0., 0., 0., 0., 0.167, 0., 0.167, 0.
, 0., 0., 0., 0.167, 0., 0., 0., 0., 0., 0.167
, 0.167, 0., 0., 0., 0., 0.167, 0.167, 0.667, 0.333, 0.5
, 0.833, 1., 0.5, 0.333, 0.667]]))

encoder = Model(inputs=input_layer, outputs=latent_layer)


In [ ]:
latent_input = Input(shape=(1,))
decoder_hidden = autoencoder.layers[-3](latent_input)  # The hidden decoder layer
decoder_hidden2 = autoencoder.layers[-2](decoder_hidden)  # The hidden decoder layer
decoder_output = autoencoder.layers[-1](decoder_hidden2)  # The output decoder layer
decoder = Model(inputs=latent_input, outputs=decoder_output)


In [ ]:

print("Input -> Latent Space -> Reconstructed Output")
for i, input_example in enumerate(input_data):
    # Get latent space value
    latent_values = encoder.predict(np.array([input_example]))
    
    # Get reconstructed output
    reconstructed_output = decoder.predict(latent_values)
    
    # Print the result
    print(f"Input: {input_example}, Latent Space: {latent_values[0]}, Reconstructed Output: {np.round(reconstructed_output[0])}")

In [ ]:
latent_slider1 = widgets.FloatSlider(value=0.0, min=-1, max=1, step=0.004, description='Latent 1', layout=widgets.Layout(width='800px'))
#latent_slider2 = widgets.FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.01, description='Latent 2')


In [ ]:
output_display = widgets.Output()


In [ ]:
def update_output(change=None):
    #latent_values = np.array([[latent_slider1.value, latent_slider2.value]])
    latent_values = np.array([[latent_slider1.value]])
    predicted_output = decoder.predict(latent_values)
    with output_display:
        output_display.clear_output(wait=True)
        print(f"Latent space: {latent_values}")
        print(f"Decoded output: {np.round(predicted_output, 2)}")


In [ ]:
latent_slider1.observe(update_output, names='value')
#latent_slider2.observe(update_output, names='value')

In [ ]:
display(latent_slider1, output_display)
#display(latent_slider1, latent_slider2, output_display)


In [ ]:
update_output()
